In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np

from climada.hazard import Hazard

tc_wind = Hazard.from_hdf5("./data/tc_wind/tc_wind_SHANSHAN_20240825000000.hdf5")
tc_wind.frequency = np.ones(len(tc_wind.frequency))/51 # change frequency to get the ensemble mean

In [17]:
import numpy as np
from climada.util.coordinates import get_country_code

idx_non_zero_wind = tc_wind.intensity.max(axis=0).nonzero()[1]

country_code_all = get_country_code(tc_wind.centroids.lat[idx_non_zero_wind], 
                                    tc_wind.centroids.lon[idx_non_zero_wind])
country_code_unique = np.trim_zeros(np.unique(country_code_all))


In [5]:
from climada.entity import ImpactFunc, ImpactFuncSet

def impf_step_func(threshold: float = 32.92):

    impf = ImpactFunc.from_step_impf((0,threshold, 100),
                                     haz_type="TC")
    
    impf_set = ImpactFuncSet()
    impf_set.append(impf)

    return(impf_set)

In [19]:
from climada.engine import ImpactCalc
from climada.util.api_client import Client
from climada.util.coordinates import country_to_iso
client = Client()

impf_set_step = impf_step_func()

imp_per_country_dict = {}

for country_code in country_code_unique:
    try:
        exp = client.get_exposures(exposures_type='litpop',
                                properties={'country_iso3num':[str(country_code)],
                                            'exponents':'(0,1)',
                                            'fin_mode':'pop',
                                            'version':'v2'
                                            }
                                )
    except client.NoResult:
        f"there is no matching dataset in Data API. Country code: {country_code}"
    
    imp_exposed = ImpactCalc(exp, impf_set_step, tc_wind).impact()

    if imp_exposed.aai_agg == 0.0:
        continue
    else:
        imp_per_country_dict[country_to_iso(country_code, "alpha3")] = imp_exposed

2024-11-26 17:11:12,081 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 1572514 coordinates.
2024-11-26 17:11:12,200 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 6905 coordinates.


In [20]:
imp_per_country_dict

{'JPN': <climada.engine.impact.Impact at 0x166462fd0>}